In [84]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import copy
import matplotlib.pyplot as plt

 ### read and modify the data

In [85]:
dataset=pd.read_table("C:/Users/Gerald/Downloads/housing_prices.txt")


In [86]:
city=copy.deepcopy(dataset['City'])
city=city.drop_duplicates()
for i in city:
    dataset[str('city='+i+'?')]=(dataset['City']==i)*1
# Normalize each column
dataset['sqft_std']=(dataset['sqft']-dataset['sqft'].mean()+0.0)/dataset['sqft'].std()
dataset['price_std']=(dataset['price']-dataset['price'].mean()+0.0)/dataset['price'].std()
dataset['bedrooms_std']=(dataset['bedrooms']-dataset['bedrooms'].mean()+0.0)/dataset['bedrooms'].std()
dataset['bath_std']=(dataset['baths']-dataset['baths'].mean()+0.0)/dataset['baths'].std()
dataset['bias']=1

In [104]:
X=copy.deepcopy(dataset)
for i in ['sqft','price','City','bedrooms','baths','price_std']:
    X=X.drop([i],axis=1)

### Split the trainset and the testset

In [105]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

### Train the model

In [106]:
def cost_function(weight):
    A=Y_train-np.dot(weight, X_train.T )
    cost=1/(2*len(X_train))*sum([i*i for i in A])
    return cost

In [107]:
def gradient(weight):
    A=Y_train-np.dot(weight, X_train.T)
    grad=np.dot(A.T, X_train)/len(X_train)
    return -grad

In [109]:
r=0.01
weight=np.zeros(52)
grad=gradient(weight)
cost1=1
cost2=0
while cost1>cost2 and np.max(gradient(weight))>0.00001:
    cost1=cost_function(weight)
    weight=weight-r*gradient(weight)
    cost2=cost_function(weight)
weight=weight-r*gradient(weight)
print(weight)

[-1.61554594 -0.33775383 -1.13030701 -0.3164122  -0.2676181   0.01222905
  0.         -0.51907089  0.          0.88454386  2.08428485  0.12913548
  0.58091655  0.         -0.36818749  0.40329437 -0.51090208 -1.05666029
 -0.74303037  1.1611613   2.1500085   0.          0.10891323  0.2777934
 -0.29820557 -0.4898905   1.08306492  1.69737926  1.15844969 -0.29103031
  1.41784951 -0.51309696 -0.2072714   0.56390604  0.69249027 -0.42341858
  0.6811468  -0.54300352 -0.31962107 -1.18978866  0.23203534  0.1838062
  0.10560482 -0.85983779 -2.09271698  0.46994237 -1.45079471 -0.47301783
  0.80117643  0.36729809 -0.28944679  0.06077376]


In [110]:
residual=Y_train-np.dot(weight, X_train.T)
MSE=sum([i*i for i in residual])/len(residual)
print(MSE)

0.0505230510166


### test

In [111]:
residual=Y_test-np.dot(weight, X_test.T)
MSE=sum([i*i for i in residual])/len(residual)
print(MSE)

0.476669250798


In [112]:
#set $500,000 as a threhold
c=(500000-dataset['price'].mean())/dataset['price'].std()
IsExp=(Y_test>c)*1

In [113]:
Is_Exp_predict=(np.dot(weight, X_test.T)>c)

In [114]:
correctRate=sum(Is_Exp_predict==IsExp)/len(IsExp)
print(correctRate)

0.96
